In [ ]:
!python -m spacy download en_core_web_md

In [2]:
import spacy
nlp = spacy.load("en_core_web_md")
    
def embed(x1):
    reference_vector = nlp(x1).vector
    return reference_vector

In [3]:
!pip install keybert


In [4]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from keybert import KeyBERT

In [6]:

kw_model = KeyBERT()

def get_topics(text, method="spacy"):
    if method == "keybert":
        topic_entities = kw_model.extract_keywords(text)  
        
    
    return topic_entities

df = pd.read_csv("fort.csv")

samples = []
for row in df.iterrows():
    row = row[1]
    sample = {
        "text": row['text_EN'],
        "topics": [row['Topic 1'].split("|")[0], row['Topic 2'].split("|")[0]]
    }
    samples.append(sample)
    

# * Scorers/Metrics
# from rouge_score import rouge_scorer
def get_vector_similarity(x1, x2):
    similarity = torch.nn.functional.cosine_similarity(x1=x1, x2=x2, dim=-1)
    return similarity


# rougeL = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

nlp = spacy.load("en_core_web_md")
    

def calculate_metric(x1,x2):
    reference_vector = nlp(x1).vector
    text_vector = nlp(x2).vector
    return get_vector_similarity(torch.tensor(reference_vector), torch.tensor(text_vector))






In [10]:
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached hdbscan-0.8.29.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl size=3542016 sha256=46fa94808936f9b99160d1a1c62d106cc5d2d2ef2c78bce5daa330c3dbf12198
  Stored in directory: /root/.cache/pip/wheels/dc/52/e3/6c6b60b126b4d5c4370cb5ac071b82950f91649d62d72f7f56
Successfully built hdbscan


In [9]:
import sklearn

In [8]:

all_keywords = set([])
for sample in tqdm(samples):
    topics = get_topics(sample["text"], method="keybert")
    sample['spacy_topics'] = [x[0] for x in topics]
    [all_keywords.add(x) for x in sample['spacy_topics']]
    # print(sample['spacy_topics'])
    
    
# print(all_keywords)

100%|██████████| 64/64 [01:08<00:00,  1.07s/it]


In [12]:
keyword_embedding_dict = {}

all_keywords = list(all_keywords) # ORDER IS IMPORTANT to get embedding 

for x in all_keywords:
    keyword_embedding_dict[x] = embed(x)

In [68]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=100)
# out = pca.fit_transform(word_values)
# print(np.sum(pca.explained_variance_ratio_))


0.9584872681455207


In [73]:
import numpy as np
from sklearn.manifold import TSNE
out = TSNE(n_components=2, learning_rate='auto',init='random', perplexity=3).fit_transform(np.array(word_values))

In [103]:
import hdbscan
word_values = list(keyword_embedding_dict.values())
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)
cluster_labels = clusterer.fit_predict(out)

In [104]:
x = out[:,0]
y = out[:,1]

In [118]:
import plotly.express as px
fig = px.scatter(x=x, y=y, color=cluster_labels, hover_name=[semantic_labels[x] for x in cluster_labels])
fig.show()

In [109]:
semantic_labels = list(label_to_semantic_topic.values())

In [116]:
import openai
openai.api_key = "sk-KEY HERE"

def get_semantic_label(keystring):
  proompt = f"The following keywords were pulled from variety of media sources. There is no guarantee they share a topic. Combine the following keywords into a general topic that balances specificity with generality. Only reply with a single, brief phrase that is the topic. Do not include any quotation marks:{keystring}"

  completion = openai.Completion.create(model="text-davinci-003", prompt=proompt)
  response = completion.choices[0].text
  filter = ''.join([chr(i) for i in range(1, 32)])
  return response.translate(str.maketrans('', '', filter))

In [117]:
label_to_semantic_topic = {}
for label in tqdm(np.unique(cluster_labels)):
  class_indeces = np.where(cluster_labels == label)
  topic_keywords = np.array(all_keywords)[class_indeces]
  if label != -1:
    keystring = " ".join(topic_keywords)
    response = get_semantic_label(keystring)
    label_to_semantic_topic[label] = response

100%|██████████| 13/13 [00:12<00:00,  1.01it/s]


In [119]:
def cluster_label_to_document(label_idx, )


{0: 'Winter Storms',
 1: 'European Wars in Eastern Europe',
 2: 'Energy Efficiency in Homes',
 3: 'Economic Crisis',
 4: ' politicsPolish Political Context',
 5: 'Monarchial Clergy',
 6: 'Political Detentions',
 7: ' LGBTQ Rights in Government',
 8: 'Education and Employment',
 9: 'Political Ideology and Revolution',
 10: 'Dietary Supplement Economics',
 11: 'Archaeological Development and Reconstruction'}